## Building GP Models from Scratch
Sometimes it is useful to build GP models outside the context of BO for data
visualization and senativity measurements, ie. learned hyperparameters. Here we
demonstrate how to build models from data outside of generators.

For this we use the 3D rosenbrock function test function.

In [1]:
# set values if testing
import os

from xopt import Xopt, Evaluator
from xopt.generators import RandomGenerator
from xopt.resources.test_functions.rosenbrock import (
    evaluate_rosenbrock,
    make_rosenbrock_vocs,
)

from xopt.generators.bayesian.visualize import visualize_model
from xopt.generators.bayesian.models.standard import StandardModelConstructor

# Ignore all warnings
import warnings

warnings.filterwarnings("ignore")


SMOKE_TEST = os.environ.get("SMOKE_TEST")
NUM_MC_SAMPLES = 1 if SMOKE_TEST else 128
NUM_RESTARTS = 1 if SMOKE_TEST else 20

# make rosenbrock function vocs in 3D
vocs = make_rosenbrock_vocs(3)

# collect some data using random sampling
evaluator = Evaluator(function=evaluate_rosenbrock)
generator = RandomGenerator(vocs=vocs)
X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)
X.random_evaluate(15)

,x0,x1,x2,y,xopt_runtime,xopt_error
0,-1.723213,1.665262,1.751155,282.388774,0.000011,False
1,-0.283855,-1.072726,-0.329373,358.027893,0.000005,False
2,-0.098538,1.882174,-0.300296,1829.366119,0.000004,False
3,-0.498350,0.460952,0.460537,13.208835,0.000003,False
4,-1.903406,-0.144903,0.010509,1429.426978,0.000003,False
5,-0.456231,1.483419,-1.230472,1342.165262,0.000003,False
6,0.188858,-0.759774,0.395774,70.321117,0.000003,False
7,1.708765,-0.641552,1.332393,1356.364601,0.000003,False
8,0.023968,0.088241,1.510258,228.294466,0.000003,False
9,0.855358,-0.304546,-0.631268,161.510389,0.000003,False


## Create GP model based on the data

In [2]:
data = X.data

In [3]:
model_constructor = StandardModelConstructor()

# here we build a model from vocs
model = model_constructor.build_model_from_vocs(vocs=vocs, data=data)

# here we build a model from info (more flexible)
model = model_constructor.build_model(
    input_names=["x0", "x1", "x2"], outcome_names=["y"], data=data
)
help(model_constructor.build_model)

Help on method build_model in module xopt.generators.bayesian.models.standard:

build_model(input_names: List[str], outcome_names: List[str], data: pandas.core.frame.DataFrame, input_bounds: Dict[str, List] = None, dtype: torch.dtype = torch.float64, device: Union[torch.device, str] = 'cpu') -> botorch.models.model_list_gp_regression.ModelListGP method of xopt.generators.bayesian.models.standard.StandardModelConstructor instance
    Construct independent models for each objective and constraint.

    Parameters
    ----------
    input_names : List[str]
        Names of input variables.
    outcome_names : List[str]
        Names of outcome variables.
    data : pd.DataFrame
        Data used for training the model.
    input_bounds : Dict[str, List], optional
        Bounds for input variables.
    dtype : torch.dtype, optional
        Data type for the model (default is torch.double).
    device : Union[torch.device, str], optional
        Device on which to perform computations (def

## Examine GP model hyperparameters
Here we look at the GP hyperparameters for the objective function (the first model).
Note: the hyperparameters here are in raw_units (due to contraints on parameter
values, ie. lengthscales > 0)

In [4]:
objective_model = model.models[vocs.output_names.index("y")]

# print raw hyperparameter values
for name, val in objective_model.named_parameters():
    print(name, val)

likelihood.noise_covar.raw_noise Parameter containing:
tensor([-20.8063], dtype=torch.float64, requires_grad=True)
mean_module.raw_constant Parameter containing:
tensor(0.7065, dtype=torch.float64, requires_grad=True)
covar_module.raw_lengthscale Parameter containing:
tensor([[0.3739, 0.2185, 1.2353]], dtype=torch.float64, requires_grad=True)


In [8]:
# print real values - note that these are in normalized coordinates
print("prior mean:", objective_model.mean_module.constant.data)
print("noise:", objective_model.likelihood.noise_covar.noise.data)
print("lengthscales", objective_model.covar_module.lengthscale.data)

prior mean: tensor(0.7065, dtype=torch.float64)
noise: tensor([0.0001], dtype=torch.float64)
lengthscales tensor([[0.3739, 0.2185, 1.2353]], dtype=torch.float64)


In [7]:
objective_model

SingleTaskGP(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (noise_prior): GammaPrior()
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (mean_module): ConstantMean()
  (covar_module): RBFKernel(
    (lengthscale_prior): LogNormalPrior()
    (raw_lengthscale_constraint): GreaterThan(2.500E-02)
  )
  (outcome_transform): Standardize()
  (input_transform): Normalize()
)

## Visualize model predictions

In [ ]:
fig, ax = visualize_model(
    model, vocs, data, variable_names=["x0", "x1"], reference_point={"x2": 0.0}
)